### Load Package：

In [2]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd

### Check Version

In [6]:
import selenium
selenium.__version__

'4.11.2'

### Load Data

In [3]:
rfilename=".\example.csv"
df=pd.read_csv(rfilename)
df.tail()

,peptide,smile
0,ACEF,C[C@H](N)C(=O)N[C@@H](CS)C(=O)N[C@@H](CCC(=O)O...
1,TCTPW,C[C@@H](O)[C@H](N)C(=O)N[C@@H](CS)C(=O)N[C@H](...
2,AGCF,C[C@H](N)C(=O)NCC(=O)N[C@@H](CS)C(=O)N[C@@H](C...
3,APDCR,C[C@H](N)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CC(=O)O)...


In [4]:
peptideseq=list(df["peptide"])
peptidesmi=list(df["smile"])

In [5]:
def inputsmiles(smiles="CC(N)C(=O)NC(C(=O)NC(Cc1ccccc1)C(=O)O)C(C)O",text_area_xpath="/html/body/div/div/div[2]/div[2]/div/div[1]/div[1]/input",button_xpath="/html/body/div/div/div[2]/div[2]/div/div[1]/div[1]/span[2]/button",a_xpath="/html/body/div/div/div[2]/div[2]/div/div[1]/div[3]/a"):
    """
    Submit smiles into table and obtained the task link
    if any error situation happen, result of link will be None
    retrun link, a string object
    """
    try:
        driver.find_element(By.XPATH,text_area_xpath).send_keys(smiles) #send the smiles
        button_click=driver.find_element(By.XPATH,button_xpath).click()
        time.sleep(1)
        link=driver.find_element(By.XPATH,a_xpath).get_attribute("href") #collect the task link
    except exception as e:
        link="None"
    return link

def loop(smileslist,seqlist,wfileNo=1,linksrecordfolder="D:\\LaverProtein\\ADMET\\tasklinks\\"):
    """
    a loop function to submit multiple smiles and collect task link
    wfileNo is the name for convenient linking to store
    linksrecordfolder is the path to stored the links
    retrun none
    """
    with open(linksrecordfolder+"tasklinks"+str(wfileNo)+".csv","w",encoding="utf8") as f:
        f.write("Peptide,tasklink\n")
        while len(smileslist) != 0:
            link=inputsmiles(smiles=smileslist.pop())    
            f.write(seqlist.pop()+","+link+"\n")
            driver.refresh()
        f.close()
    return None

def downloadcsv(driver,rfileNo=1,folder="C:\\Users\\ff\\Downloads\\",linksrecordfolder="D:\\LaverProtein\\ADMET\\tasklinks\\",resultwritefolder="D:\\LaverProtein\\ADMET\\"):
    """
    dowmload the csv file based on the built link
    folder is the default download point of brower
    linksrecordfolder is the point to stored the links which were generated in last step
    resultwritefolder is the point to stored the admet result
    return none
    """
    download_button_xpath="/html/body/div/div/div[2]/div/div/div[2]/div/div[1]/div[2]/a[1]"
    tbodytag1="/html/body/div/div/div[2]/div/div/div[2]/div/div[3]/div[3]/table/tbody"
    tbodytag2="/html/body/div/div/div[2]/div/div/div[2]/div/div[4]/div[3]/table/tbody"
    
    df=pd.read_csv(linksrecordfolder+"tasklinks"+str(rfileNo)+".csv") #read the link data
    
    links = [link.replace("\n","") for link in list(df["tasklink"])]  #convert to list
    pepseq=list(df["Peptide"])
     
    for link in range(len(links)):
        state = True
        while state:
            try:
                driver.get(links[link]) 
                time.sleep(1.5)
                classificationpre=driver.find_element(By.XPATH,tbodytag1).text
                regressionpre=driver.find_element(By.XPATH,tbodytag2).text
                driver.refresh()
                d=classificationpre.split("\n")+regressionpre.split('\n')
                string,count="",0
                with open(resultwritefolder+pepseq[link]+".txt","w+",encoding="utf8") as f:
                    for i in range(1,len(d)+1):
                        i=d[i-1]+','
                        string+=i
                        count+=1
                        if count/3 == 1:
                            f.write(string+"\n")
                            string,count="",0
                    f.close()
                state = False
            except:
                print(links[link]+" is error.")
                state = False
    return None      
            

def preparefun():
    """
    a test function
    """
    link="http://lmmd.ecust.edu.cn/admetsar2/result/?tid=709657"
    tbodytag1="/html/body/div/div/div[2]/div/div/div[2]/div/div[3]/div[3]/table/tbody"
    tbodytag2="/html/body/div/div/div[2]/div/div/div[2]/div/div[4]/div[3]/table/tbody"
    driver.get(link) 
    time.sleep(2)
    classificationpre=driver.find_element(By.XPATH,tbodytag1).text
    regressionpre=driver.find_element(By.XPATH,tbodytag2).text
    return classificationpre,regressionpre   



### Input Data:

In [6]:
if __name__ == "__main__":
    linkStoredplace="C:\\Users\\ff\\"
    print("Note! the task links text will be stored in path "+linkStoredplace)
    driver=webdriver.Firefox()
    driver.implicitly_wait(10) #aviod unable to locate the elements
    driver.set_window_size(580,840)
    driver.get("http://lmmd.ecust.edu.cn/admetsar2/")
    loop(peptidesmi,peptideseq,wfileNo=1,linksrecordfolder=linkStoredplace) #submit the task
    print("All task were submitted, you could take a break and come back to dowmload them later.")
    print("Run the code below to download the result.")

Note! the task links text will be stored in path C:\Users\ff\
All task were submitted, you could take a break and come back to dowmload them later.
Run the code below to download the result.


### Download Data:

In [9]:
if __name__ == "__main__":
    url="http://lmmd.ecust.edu.cn/admetsar2/"
    driver=webdriver.Firefox()
    driver.implicitly_wait(10)
    driver.set_window_size(580,840)
    lsf="C:\\Users\\ff\\"
    rwf="C:\\Users\\ff\\"
    ddf="C:\\Users\\ff\\Downloads\\"
    print("Note! the link text was stored in "+lsf)
    print("Note! the result will stored in "+rwf)
    print("Note! the default saving path of browser is "+ddf)
    #开始下载任务：
    downloadcsv(driver,rfileNo=1,folder=ddf,linksrecordfolder=lsf,resultwritefolder=rwf)
    driver.close()  ##关闭当前的窗口

Note! the link text was stored in C:\Users\ff\
Note! the result will stored in C:\Users\ff\
Note! the default saving path of browser is C:\Users\ff\Downloads\
http://lmmd.ecust.edu.cn/admetsar2/result/?tid=913797 is error.
http://lmmd.ecust.edu.cn/admetsar2/result/?tid=913798 is error.
http://lmmd.ecust.edu.cn/admetsar2/result/?tid=913799 is error.
http://lmmd.ecust.edu.cn/admetsar2/result/?tid=913800 is error.
